In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth
!pip install snac


In [ ]:
from unsloth import FastLanguageModel
import torch
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.
# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Karayakar/Orpheus-TTS-Turkish-PT-5000",
    max_seq_length= 2048, # Choose any for long context!
    dtype = dtype,
    load_in_4bit = load_in_4bit,

    #token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.4.7: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/21.2k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.32G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/5.64M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/534 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/22.8M [00:00<?, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 64, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 64,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.4.7 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [ ]:
from datasets import load_dataset
dataset = load_dataset("/content/drive/MyDrive/youtube_trancript/huggingface_dataset", split = "train")

Repo card metadata block was not found. Setting CardData to empty.


Resolving data files:   0%|          | 0/28 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Loading dataset shards:   0%|          | 0/29 [00:00<?, ?it/s]

In [ ]:
import locale
import torchaudio.transforms as T
import os
import torch
from snac import SNAC

# Locale ayarlaması
locale.getpreferredencoding = lambda: "UTF-8"
ds_sample_rate = dataset[0]["audio"]["sampling_rate"]

# SNAC modelini yükleyip CUDA'ya taşıyalım
snac_model = SNAC.from_pretrained("hubertsiuzdak/snac_24khz")
snac_model = snac_model.to("cuda")

def tokenise_audio(waveform):
    # Dalga formunu tensor haline getiriyoruz
    waveform = torch.from_numpy(waveform).unsqueeze(0)
    waveform = waveform.to(dtype=torch.float32)
    # Ses örnekleme oranını 24000 Hz'e düşürüyoruz
    resample_transform = T.Resample(orig_freq=ds_sample_rate, new_freq=24000)
    waveform = resample_transform(waveform)

    # Tekrar boyut ekleyip CUDA'ya taşıyoruz
    waveform = waveform.unsqueeze(0).to("cuda")

    # SNAC ile kodları üretiyoruz
    with torch.inference_mode():
        codes = snac_model.encode(waveform)

    all_codes = []
    for i in range(codes[0].shape[1]):
        all_codes.append(codes[0][0][i].item() + 128266)
        all_codes.append(codes[1][0][2*i].item() + 128266 + 4096)
        all_codes.append(codes[2][0][4*i].item() + 128266 + (2*4096))
        all_codes.append(codes[2][0][(4*i)+1].item() + 128266 + (3*4096))
        all_codes.append(codes[1][0][(2*i)+1].item() + 128266 + (4*4096))
        all_codes.append(codes[2][0][(4*i)+2].item() + 128266 + (5*4096))
        all_codes.append(codes[2][0][(4*i)+3].item() + 128266 + (6*4096))
    return all_codes

def add_codes(example):
    # Varsayılan olarak codes_list None başlatılıyor
    codes_list = None
    try:
        answer_audio = example.get("audio")
        # Eğer geçerli bir audio array varsa tokenisation yapılıyor
        if answer_audio and "array" in answer_audio:
            audio_array = answer_audio["array"]
            codes_list = tokenise_audio(audio_array)
    except Exception as e:
        print(f"Skipping row due to error: {e}")
    example["codes_list"] = codes_list
    return example

# Audio içeren örnekleri dönüştürüp "audio" sütununu kaldırıyoruz
dataset = dataset.map(add_codes, remove_columns=["audio"])

# Özel token ayarları
tokeniser_length = 128256
start_of_text = 128000
end_of_text = 128009

start_of_speech = tokeniser_length + 1
end_of_speech = tokeniser_length + 2

start_of_human = tokeniser_length + 3
end_of_human = tokeniser_length + 4

start_of_ai = tokeniser_length + 5
end_of_ai = tokeniser_length + 6
pad_token = tokeniser_length + 7

audio_tokens_start = tokeniser_length + 10

# codes_list içeren ve boş olmayan örnekleri filtreleyelim
dataset = dataset.filter(lambda x: x["codes_list"] is not None)
dataset = dataset.filter(lambda x: len(x["codes_list"]) > 0)

def remove_duplicate_frames(example):
    vals = example["codes_list"]
    if len(vals) % 7 != 0:
        raise ValueError("Input list length must be divisible by 7")
    result = vals[:7]
    removed_frames = 0
    for i in range(7, len(vals), 7):
        current_first = vals[i]
        previous_first = result[-7]
        if current_first != previous_first:
            result.extend(vals[i:i+7])
        else:
            removed_frames += 1
    example["codes_list"] = result
    return example

dataset = dataset.map(remove_duplicate_frames)

# Bilgilendirme mesajı (prompt üzerinde düzenleme yapabilirsiniz)
tok_info = '''*** HERE you can modify the text prompt
If you are training a multi-speaker model (e.g., canopylabs/orpheus-3b-0.1-ft),
ensure that the dataset includes a "source" field and format the input accordingly:
- Single-speaker: f"{example['text']}"
- Multi-speaker: f"{example['source']}: {example['text']}"
'''
print(tok_info)

# Tokenizer üzerinde padding için bir token tanımlayalım (eğer yoksa)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token  # veya uygun başka bir token

def create_input_ids(example):
    # "text" sütunu varsa bunu kullanıyoruz; yoksa varsayılan metin kullanılır.
    text_value = example.get("text", None)
    if text_value is None:
        text_prompt = "dummy text"  # Varsayılan metin, ihtiyaca göre düzenlenebilir.
    else:
        text_prompt = f"{example['source']}: {text_value}" if "source" in example else text_value

    text_ids = tokenizer.encode(text_prompt, add_special_tokens=True)
    text_ids.append(end_of_text)

    # Parçaları ayrı ayrı tanımlıyoruz:
    # human_prefix: sabit metin tokenları (başlangıç, metin, bitiş gibi)
    human_prefix = [start_of_human] + text_ids + [end_of_human] + [start_of_ai] + [start_of_speech]
    # audio_tokens: ses tokenları (codes_list varsa, yoksa boş liste)
    audio_tokens = example.get("codes_list", [])
    # human_suffix: sonlandırma tokenları
    human_suffix = [end_of_speech] + [end_of_ai]

    # Maksimum sequence uzunluğu
    max_seq_length = 2048

    # Sabit kısımların toplam uzunluğu
    fixed_length = len(human_prefix) + len(human_suffix)

    # Eğer sabit kısım çok uzun ise, sadece metin kısmını keserek işlem yapıyoruz.
    if fixed_length > max_seq_length:
        truncated_prefix = human_prefix[:max_seq_length - len(human_suffix)]
        input_ids = truncated_prefix + human_suffix
    else:
        # Ses tokenları için kalan alan:
        allowed_audio_tokens = max_seq_length - fixed_length
        if len(audio_tokens) > allowed_audio_tokens:
            audio_tokens = audio_tokens[:allowed_audio_tokens]
        input_ids = human_prefix + audio_tokens + human_suffix

    example["input_ids"] = input_ids
    example["labels"] = input_ids  # loss hesaplaması için aynı dizi kullanılıyor
    example["attention_mask"] = [1] * len(input_ids)

    return example

# İlk olarak, create_input_ids dönüşümünü "text" ve "codes_list" sütunlarını kaldırmadan uygulayalım
# (Eğer bu sütunlar mevcutsa, fonksiyon onları kullanacak. Aksi halde create_input_ids içinde varsayılan değer kullanılacak.)
dataset = dataset.map(create_input_ids, remove_columns=["text", "codes_list"])

# Eğitim için sadece gerekli sütunları tutalım:
columns_to_keep = ["input_ids", "labels", "attention_mask"]
columns_to_remove = [col for col in dataset.column_names if col not in columns_to_keep]
dataset = dataset.remove_columns(columns_to_remove)

# Örnek bir batch üzerinden test edelim:
sample = dataset[0]
print("Input IDs length:", len(sample["input_ids"]))
assert len(sample["input_ids"]) <= 2048, "Input IDs length exceeds 2048"

# Eğer isterseniz, tekrar create_input_ids dönüşümünü uygulamayı deneyebilirsiniz,
# ancak sütunlar zaten işlendiyse tekrardan uygulamaya gerek yoktur.
# Aşağıdaki satırı yorum satırı yaparak devre dışı bırakabilirsiniz:
# dataset = dataset.map(create_input_ids)

# Son olarak, veri setindeki sütunları yazdıralım:
print("Dataset sütunları:", dataset.column_names)


config.json:   0%|          | 0.00/300 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/79.5M [00:00<?, ?B/s]

Parameter 'function'=<function add_codes at 0x7c8dca125120> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/23925 [00:00<?, ? examples/s]

Filter:   0%|          | 0/23925 [00:00<?, ? examples/s]

Filter:   0%|          | 0/23925 [00:00<?, ? examples/s]

Map:   0%|          | 0/23925 [00:00<?, ? examples/s]

*** HERE you can modify the text prompt
If you are training a multi-speaker model (e.g., canopylabs/orpheus-3b-0.1-ft),
ensure that the dataset includes a "source" field and format the input accordingly:
- Single-speaker: f"{example['text']}"
- Multi-speaker: f"{example['source']}: {example['text']}"



Map:   0%|          | 0/23925 [00:00<?, ? examples/s]

Input IDs length: 131
Dataset sütunları: ['input_ids', 'labels', 'attention_mask']


In [ ]:
from transformers import TrainingArguments, Trainer, DataCollatorForSeq2Seq

trainer = Trainer(
    model=model,
    train_dataset=dataset,
    args=TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        learning_rate=2e-4,
        fp16=True,   # Tesla T4, FP16 desteği sağlıyor.
        bf16=False,  # BF16 desteği olmadığı için False.
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="/content/drive/MyDrive/tts_fine_tune/models/sesame_cengizhan",
        report_to="none",  # WandB vs. raporlama istemiyorsanız.
    ),
)
import torch

# Dataset'ten bir örnek alalım. (Gerekirse batch haline getirin)
example = dataset[0]
# Batch boyutu ekleyelim
input_ids = torch.tensor([example["input_ids"]]).to("cuda")
labels = torch.tensor([example["labels"]]).to("cuda")

with torch.no_grad():
    outputs = model(input_ids)

print("Logits shape:", outputs.logits.shape)  # Beklenen: (batch_size, seq_length, vocab_size)
print("Labels shape:", labels.shape)           # Beklenen: (batch_size, seq_length)



Logits shape: torch.Size([1, 131, 156940])
Labels shape: torch.Size([1, 131])


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 23,925 | Num Epochs = 3 | Total steps = 17,943
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 4 x 1) = 4
 "-____-"     Trainable parameters = 97,255,424/1,988,836,352 (4.89% trained)


Step,Training Loss
1,7.347300
2,7.306500
3,7.446100
4,7.137000
5,7.348400
6,6.990800
7,6.899300
8,6.864300
9,6.425800
10,6.510400


Unsloth: Will smartly offload gradients to save VRAM!


In [ ]:
prompts = [
    " Şimdi, bu modelin heyecanla denemesini yapıyorum ve çok güzel bir cevap geleceğine inanıyorum yaşasın konyaspor ",
]

chosen_voice = None # None for single-speaker

In [ ]:
#@title Run Inference


FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# Moving snac_model cuda to cpu
snac_model.to("cpu")

prompts_ = [(f"{chosen_voice}: " + p) if chosen_voice else p for p in prompts]

all_input_ids = []

for prompt in prompts_:
  input_ids = tokenizer(prompt, return_tensors="pt").input_ids
  all_input_ids.append(input_ids)

start_token = torch.tensor([[ 128259]], dtype=torch.int64) # Start of human
end_tokens = torch.tensor([[128009, 128260]], dtype=torch.int64) # End of text, End of human

all_modified_input_ids = []
for input_ids in all_input_ids:
  modified_input_ids = torch.cat([start_token, input_ids, end_tokens], dim=1) # SOH SOT Text EOT EOH
  all_modified_input_ids.append(modified_input_ids)

all_padded_tensors = []
all_attention_masks = []
max_length = max([modified_input_ids.shape[1] for modified_input_ids in all_modified_input_ids])
for modified_input_ids in all_modified_input_ids:
  padding = max_length - modified_input_ids.shape[1]
  padded_tensor = torch.cat([torch.full((1, padding), 128263, dtype=torch.int64), modified_input_ids], dim=1)
  attention_mask = torch.cat([torch.zeros((1, padding), dtype=torch.int64), torch.ones((1, modified_input_ids.shape[1]), dtype=torch.int64)], dim=1)
  all_padded_tensors.append(padded_tensor)
  all_attention_masks.append(attention_mask)

all_padded_tensors = torch.cat(all_padded_tensors, dim=0)
all_attention_masks = torch.cat(all_attention_masks, dim=0)

input_ids = all_padded_tensors.to("cuda")
attention_mask = all_attention_masks.to("cuda")
generated_ids = model.generate(
      input_ids=input_ids,
      attention_mask=attention_mask,
      max_new_tokens=1200,
      do_sample=True,
      temperature=0.6,
      top_p=0.95,
      repetition_penalty=1.1,
      num_return_sequences=1,
      eos_token_id=128258,
     use_cache = True
  )
token_to_find = 128257
token_to_remove = 128258

token_indices = (generated_ids == token_to_find).nonzero(as_tuple=True)

if len(token_indices[1]) > 0:
    last_occurrence_idx = token_indices[1][-1].item()
    cropped_tensor = generated_ids[:, last_occurrence_idx+1:]
else:
    cropped_tensor = generated_ids

mask = cropped_tensor != token_to_remove

processed_rows = []

for row in cropped_tensor:
    masked_row = row[row != token_to_remove]
    processed_rows.append(masked_row)

code_lists = []

for row in processed_rows:
    row_length = row.size(0)
    new_length = (row_length // 7) * 7
    trimmed_row = row[:new_length]
    trimmed_row = [t - 128266 for t in trimmed_row]
    code_lists.append(trimmed_row)


def redistribute_codes(code_list):
  layer_1 = []
  layer_2 = []
  layer_3 = []
  for i in range((len(code_list)+1)//7):
    layer_1.append(code_list[7*i])
    layer_2.append(code_list[7*i+1]-4096)
    layer_3.append(code_list[7*i+2]-(2*4096))
    layer_3.append(code_list[7*i+3]-(3*4096))
    layer_2.append(code_list[7*i+4]-(4*4096))
    layer_3.append(code_list[7*i+5]-(5*4096))
    layer_3.append(code_list[7*i+6]-(6*4096))
  codes = [torch.tensor(layer_1).unsqueeze(0),
         torch.tensor(layer_2).unsqueeze(0),
         torch.tensor(layer_3).unsqueeze(0)]

  # codes = [c.to("cuda") for c in codes]
  audio_hat = snac_model.decode(codes)
  return audio_hat

my_samples = []
for code_list in code_lists:
  samples = redistribute_codes(code_list)
  my_samples.append(samples)
from IPython.display import display, Audio
if len(prompts) != len(my_samples):
  raise Exception("Number of prompts and samples do not match")
else:
  for i in range(len(my_samples)):
    print(prompts[i])
    samples = my_samples[i]
    display(Audio(samples.detach().squeeze().to("cpu").numpy(), rate=24000))
# Clean up to save RAM
del my_samples,samples

 Şimdi, bu modelin heyecanla denemesini yapıyorum ve çok güzel bir cevap geleceğine inanıyorum yaşasın konyaspor 


In [ ]:
model.save_pretrained("/content/drive/MyDrive/tts_fine_tune/models/mevlut_emir_tts")  # Local saving
tokenizer.save_pretrained("/content/drive/MyDrive/tts_fine_tune/models/mevlut_emir_tts")


('/content/drive/MyDrive/tts_fine_tune/models/mevlut_emir_tts/tokenizer_config.json',
 '/content/drive/MyDrive/tts_fine_tune/models/mevlut_emir_tts/special_tokens_map.json',
 '/content/drive/MyDrive/tts_fine_tune/models/mevlut_emir_tts/tokenizer.json')